In [ ]:
import numpy as np
from tensorflow.keras import regularizers
import math
import tensorflow as tf
import tensorflow.keras.layers as tfl
from keras import backend as K

In [ ]:
from tensorflow.keras.datasets import cifar100

(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='coarse')

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 416s 2us/step


In [ ]:
print(X_train.shape,y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [ ]:
import cv2
X_train = np.array([cv2.resize(img, (140, 140)) for img in X_train])

In [ ]:
X_test = np.array([cv2.resize(img, (140, 140)) for img in X_test])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
y_train=enc.fit_transform(y_train).toarray().astype(int)
y_test=enc.transform(y_test).toarray().astype(int)


print(y_train.shape)
print(y_train[0])

(50000, 20)
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [ ]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(tfl.RandomFlip('horizontal'))
    data_augmentation.add(tfl.RandomRotation(0.2))

    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()

In [ ]:
def arch(input_shape, data_augmentation):

    input_img = tf.keras.Input(shape=input_shape)
    layer = data_augmenter()(input_img)
    layer=tfl.Conv2D(filters= 96 , kernel_size= 11,strides=(4, 4))(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Conv2D(filters= 256 , kernel_size= 5 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 256 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Flatten()(layer)

    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=1000, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    outputs=tfl.Dense(units= 20 , activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

In [ ]:
conv_model = arch((140, 140, 3),data_augmentation)
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 140, 140, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 140, 140, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 33, 33, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 33, 33, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 17, 17, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 17, 17, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 17, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 17, 17, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 9, 9, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 9, 9, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 9, 9, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 9, 9, 256)      │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5, 5, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    26,218,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,31

 Total params: 50,903,228 (194.18 MB)

 Trainable params: 50,883,628 (194.11 MB)

 Non-trainable params: 19,600 (76.56 KB)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)
history = conv_model.fit(train_dataset,epochs=10,validation_data=test_dataset,batch_size=64,shuffle=True)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 675s 857ms/step - accuracy: 0.1434 - loss: 2.9833 - val_accuracy: 0.2158 - val_loss: 3.0546
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 641s 820ms/step - accuracy: 0.2603 - loss: 2.4502 - val_accuracy: 0.2539 - val_loss: 2.5970
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 634s 811ms/step - accuracy: 0.3192 - loss: 2.2142 - val_accuracy: 0.3671 - val_loss: 2.0829
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 633s 809ms/step - accuracy: 0.3608 - loss: 2.0593 - val_accuracy: 0.3293 - val_loss: 2.3116
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 742s 949ms/step - accuracy: 0.3964 - loss: 1.9528 - val_accuracy: 0.3549 - val_loss: 2.1497
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 821s 1s/step - accuracy: 0.4288 - loss: 1.8324 - val_accuracy: 0.4488 - val_loss: 1.7954
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 634s 811ms/step - accuracy: 0.4576 - loss: 1.7261 - val_accuracy: 0.3365 - val_loss: 2.3058
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 619s 791ms/step - accuracy: 0.4713 - lo